In [85]:
import pandas as pd
import wget
import requests
import os
from bs4 import BeautifulSoup

## Reading in the file

In [33]:
mainFile = pd.read_csv("CIK_File.csv")
mainFile["CIK"] = mainFile["CIK"].astype(str).apply(lambda x : x.zfill(10))
companyCIK = list(set(mainFile["CIK"]))

## Retrieving information from SEC webpages

In [63]:
corpInfo = {}
for CIK in companyCIK:
    SEC_page = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=" + \
                CIK + "&type=10-k&dateb=&owner=exclude&count=40"
    searchYear = list(mainFile[mainFile["CIK"] == CIK]["FYEAR"])
    read_SEC_page = requests.get(SEC_page)
    SEC_parsed_page = BeautifulSoup(read_SEC_page.text, "html.parser")
    SEC_table = SEC_parsed_page.find("table", {"class" : "tableFile2"})
    SEC_table_schema = []
    for SEC_row in SEC_table.find_all("tr"):
        if SEC_row.find_all("td"):
            holder = {}
            SEC_col = SEC_row.find_all("td")
            holder["fileType"] = SEC_col[0].text
            holder["documentsLinks"] = "https://www.sec.gov" + SEC_col[1].find("a", {"id" : "documentsbutton"}, href = True)["href"]
            holder["issueDate"] = SEC_col[3].text
            realYear = int(holder["issueDate"].split("-")[0]) - 1
            if realYear in searchYear:
                holder["realYear"] = realYear
                SEC_table_schema.append(holder)
    corpInfo[CIK] = SEC_table_schema

## Starting download process

In [96]:
for CIK in companyCIK:
    os.mkdir("tenK/" + CIK)
    for info_list in corpInfo[CIK]:
        if info_list["fileType"] == "10-K":
            ten_k_page = requests.get(info_list["documentsLinks"])
            parsed_ten_k_page = BeautifulSoup(ten_k_page.text, "html.parser")
            ten_k_rows = parsed_ten_k_page.find("table", {"summary" : "Document Format Files"}).findAll("tr")
            for row in ten_k_rows:
                if row.find_all("td"):
                    if row.find_all("td")[1].text == "Complete submission text file":
                        ten_k_Link = "https://www.sec.gov/" + row.find_all("td")[2].find(href = True)["href"]
            extension = ten_k_Link.split(".")[-1]
            downloadPath = "tenK/" + CIK + "/" + CIK + "_" + str(info_list["realYear"]) + "." + extension
            wget.download(ten_k_Link, downloadPath)